In [2]:
import os, sys

notebook_path = os.path.dirname(os.path.abspath("__file__"))
# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
basepath = os.path.dirname(os.path.dirname(notebook_path))

sys.path.append(core_path)
sys.path.append(basepath)

from core.klusta_functions import klusta
from core.klusta_utils import session_analyzable, find_tetrodes
from core.Tint_Matlab import get_setfile_parameter
from core.defaultParameters import get_default_settings, calculateUseFeatures

# Define Settings

In [3]:
settings = get_default_settings()

# the default naming structure is the directory\session_basename_tetrodeNumber.cut
# for example: C:\Example\ExampleSession_1.cut for tetrode 1.
# However, there may be a cause where you want to re-analyze with different cut file nomenclature
# so we will allow you to append a value to the end to produce the following:
# directory\session_basename[appendValue]_tetrodeNumber.cut
# so if we append 'TEST' the output would be: C:\Example\ExampleSessionTEST_1.cut for tetrode 1
# set the cut_append value to None for the default
cut_append='KK'  # custom append value
# cut_append=None  # uncomment this for default value

# channels to use:
settings['1'] = 1  # defaults to 1, which is on
settings['2'] = 1  # defaults to 1, which is on
settings['3'] = 1  # defaults to 1, which is on
settings['4'] = 1  # defaults to 1, which is on

# features to use:
settings['PC1'] = 1  # defaults to 1, which is on
settings['PC2'] = 1  # defaults to 1, which is on
settings['PC3'] = 1  # defaults to 1, which is on
settings['PC4'] = 0  # defaults to 0, which is off
settings['A'] = 0  # defaults to 0, which is off
settings['Vt'] = 0  # defaults to 0, which is off
settings['P'] = 0  # defaults to 0, which is off
settings['T'] = 0  # defaults to 0, which is off
settings['tP'] = 0  # defaults to 0, which is off
settings['tT'] = 0  # defaults to 0, which is off
settings['En'] = 0  # defaults to 0, which is off
settings['Ar'] = 0  # defaults to 0, which is off

# re-caculating the feature masks (if channel 2 is off, we will make sure to ignore
# those features)
UseFeatures, numFeats = calculateUseFeatures(settings)
settings['UseFeatures'] = UseFeatures  # feature mask
settings['NumFeat'] = numFeats  # number of features used

# Advanced KlustaKwik Parameters
settings['MaxPos'] = 30  # default 3
settings['nStarts'] = 1  # default = 1
settings['RandomSeed'] = 1  # default = 1
settings['DistThresh'] = 6.907755  # default is 6.907755
settings['FullStepEvery'] = 20  # default is 20
settings['ChangedThresh'] = 0.05  # default is 0.05
settings['MaxIter'] = 500  # default is 500
settings['SplitEvery'] = 40  # default is 40
settings['Subset'] = 1 # default is 1
settings['PenaltyK'] = 1.0  # default is 1.0
settings['PenaltyKLogN'] = 0.0  # default is 0.0

# Reporting Values
settings['Verbose'] = 1  # default of 1
settings['Screen'] = 1  # default of 1
settings['Log File'] = 1  # default of 1


# Misc Settings
settings['Silent'] = 0   # the KK windows will pop up
# number of simultaneous threads to analyze, defaults to 1, uncomment to override
settings['NumThreads'] = 4  # default is 1
settings['Cores'] = os.cpu_count()

# Experimenter Settings
experimenter_settings = {
    'Geoff' : 'geoffrey.m.barrett@gmail.com' # can do [email1@.., email2@..] if you want it sent to more than 1
}

# Email Settings
smtp_settings = {}
smtp_settings['Notification'] = 0 #  1 for send e-mails, 0 for don't send

# if you have the notifications set to 0 you don't have to worry about this.
# we will need an e-mail to send these experimenter's e-mails from
smtp_settings['Username'] = 'example@gmail.com'
smtp_settings['Password'] = 'password'  # associated password
smtp_settings['ServerName'] = 'smtp.gmail.com' # the smtp server name, 'smtp.gmail.com' for gmail
smtp_settings['Port'] = 587  # 587 default for gmail

# Analyze Directory

In [4]:
batch = False

if batch:
    settings['nonbatch'] = 0
else:
    settings['nonbatch'] = 1

settings['MoveFiles'] = 0  # set to 1 if you want to move to "processed" folder
    
directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\ANT1_2\ParallelLinearGlobalTrack'
# directory = r'C:\Users\Taub Institute\Desktop\test_batchtint'

# Finds Sessions

In [147]:
if batch:
    sub_directories = [d for d in os.listdir(directory)
                                   if os.path.isdir(os.path.join(directory, d)) and
                                   d not in ['Processed', 'Converted']]  # finds the subdirectories within each folder

else:
    sub_directories = [os.path.basename(directory)]
    directory = os.path.dirname(directory)

set_files = []
if len(sub_directories) > 0:
    [[set_files.append(os.path.join(directory, d, file)) for file in os.listdir(os.path.join(directory, d))
                             if '.set' in file and
                             not os.path.isdir(os.path.join(directory, d, file))] for d in sub_directories]

In [148]:
print('There are %d total set files found!' % (len(set_files)))

There are 17 total set files found!


In [149]:
# checking that the set files are analyzable (have the appropriate files, or don't have
# existing cut values already. Essentially all it needs are the tetrode files.
if set_files:
    set_files = [file for file in set_files if session_analyzable(os.path.dirname(file), 
                                                             os.path.splitext(
                                                                 os.path.basename(file))[0],
                                                            append=cut_append)]

In [150]:
print('There are %d un-analyzed set files found!' % (len(set_files)))

There are 10 un-analyzed set files found!


In [151]:
analyzed_set_files = klusta(set_files, settings, 
       smtp_settings=smtp_settings, 
       experimenter_settings=experimenter_settings,
                           append=cut_append)

[2019-04-03 14:15:28]: There are 10 '.set' files to analyze!
[2019-04-03 14:15:28]: Now analyzing tetrodes associated with the ANT_1_2_1075_plgt_190314_145652_ms '.set' file (1/10)!
[2019-04-03 14:15:28]: Now analyzing the following file: ANT_1_2_1075_plgt_190314_145652_ms.1!
[2019-04-03 14:15:28]: Now analyzing the following file: ANT_1_2_1075_plgt_190314_145652_ms.2!
[2019-04-03 14:15:28]: Now analyzing the following file: ANT_1_2_1075_plgt_190314_145652_ms.3!
[2019-04-03 14:15:28]: Now analyzing the following file: ANT_1_2_1075_plgt_190314_145652_ms.4!
[2019-04-03 14:16:58]: The analysis of the ANT_1_2_1075_plgt_190314_145652_ms.3 file has finished!
[2019-04-03 14:17:53]: The analysis of the ANT_1_2_1075_plgt_190314_145652_ms.2 file has finished!
[2019-04-03 14:18:23]: The analysis of the ANT_1_2_1075_plgt_190314_145652_ms.1 file has finished!
[2019-04-03 14:19:13]: The analysis of the ANT_1_2_1075_plgt_190314_145652_ms.4 file has finished!
[2019-04-03 14:19:13]: Now analyzing the f

[2019-04-03 14:44:57]: The analysis of the ant_1_2_1175_plgt_190321_125429_ms.7 file has finished!
[2019-04-03 14:45:07]: The analysis of the ant_1_2_1175_plgt_190321_125429_ms.6 file has finished!
[2019-04-03 14:45:07]: Now analyzing tetrodes associated with the ant_1_2_1175_plgt_190321_141722_ms '.set' file (6/10)!
[2019-04-03 14:45:07]: Now analyzing the following file: ant_1_2_1175_plgt_190321_141722_ms.1!
[2019-04-03 14:45:07]: Now analyzing the following file: ant_1_2_1175_plgt_190321_141722_ms.2!
[2019-04-03 14:45:07]: Now analyzing the following file: ant_1_2_1175_plgt_190321_141722_ms.3!
[2019-04-03 14:45:07]: Now analyzing the following file: ant_1_2_1175_plgt_190321_141722_ms.4!
[2019-04-03 14:46:15]: The analysis of the ant_1_2_1175_plgt_190321_141722_ms.4 file has finished!
[2019-04-03 14:46:41]: The analysis of the ant_1_2_1175_plgt_190321_141722_ms.2 file has finished!
[2019-04-03 14:47:58]: The analysis of the ant_1_2_1175_plgt_190321_141722_ms.3 file has finished!
[201

[2019-04-03 15:09:34]: The analysis of the ant_1_2_1175_plgt_190325_151247_ms.6 file has finished!
[2019-04-03 15:10:34]: The analysis of the ant_1_2_1175_plgt_190325_151247_ms.8 file has finished!
[2019-04-03 15:11:10]: The analysis of the ant_1_2_1175_plgt_190325_151247_ms.7 file has finished!
[2019-04-03 15:11:10]: Analysis for the following directory has been completed: E:\Apollo_D_Drive\data\VirtualMazeData\ANT1_2\ParallelLinearGlobalTrack!


In [145]:
def get_temp_files(set_path, append=None):

    append = cut_append
    if append is None:
        ini_filename_value = str('Filename="%s"' % set_path)
    else:
        ini_filename_value = str('Filename="%s%s"' % (set_path, append))

    ini_filename_value = str('Filename="%s%s"' % (set_path, append))

    directory = os.path.dirname(set_path)
    
    basename = os.path.basename(set_path)
    if append is not None:
         basename += append
    
    file_list = os.listdir(directory)

    temp_files = [os.path.join(directory, file) for file in file_list if basename in file and is_temp_ext(file)]
    
    return temp_files

def ext_found(filename, ex):
    if file.find(ex) != -1:
        return True
    return False

def is_temp_ext(filename):
    
    extensions = ['.temp.clu', '.klg', '.initialclusters.', '.fmask.', '.fet.']
    
    if any(ext_found(filename, ex) for ex in extensions):
        for ex in extensions:
            if ex in filename:
                filename = filename[filename.find(ex):]
                if any(ex == x for x in ['.temp.clu', '.klg', '.fmask.', '.fet.']):
                    try:
                        int(os.path.splitext(filename[1:])[-1][1:])
                        return True
                    except ValueError:
                        pass
                elif '.initialclusters.' in ex:
                    filename = filename.split('.')
                    try:
                        int(filename[2])
                        int(filename[4])
                        return True
                    except ValueError:
                        pass
    return False
            

In [146]:
delete_temp = True

if delete_temp:
    
    for file in set_files:
        set_path = os.path.splitext(set_files[0])[0]
        temp_files = get_temp_files(set_path, append=cut_append)
        for _f in temp_files:
            os.remove(_f)